<a href="https://colab.research.google.com/github/terrancemcgee/API-Forecast-Stocks/blob/main/dark_dark_option_%24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install discord-webhook

!pip install python-telegram-bot --upgrade

import os
import datetime
import time
import math
import numpy as np
import pandas as pd
import requests
from statistics import mean
import matplotlib.pyplot as plt

from discord_webhook import DiscordWebhook
from telegram import Bot
from oauth2client.service_account import ServiceAccountCredentials
import gspread

# ---------------------------------
# CONFIGURATION & AUTHENTICATION
# ---------------------------------

ALPHA_VANTAGE_API_KEY = "WT6ZWUGNW2BG492N"

# Telegram & Discord
TELEGRAM_BOT_TOKEN = "7454430837:AAHFtVI3WJ-eG7UBUxAxGx6jkC4IfcghjQg"
TELEGRAM_CHAT_ID = "1001989070467"
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1309246268135968789/dRUdyMUp-wqlZGKHJ0LVeg3F-TkykW31vzfEgtAsIo4IYACmR_fB1LhZQmgccN-Xcu-4"
telegram_bot = Bot(token=TELEGRAM_BOT_TOKEN)

# Google Sheets
CREDS_FILE = os.path.join(os.path.expanduser('~'), 'dark_roast_strategy', 'dark_roast_strategy', 'credentials', 'darkroastoption-df4b2540117.json')
SHEET_NAME = '18QnDj_eZplfZvEjjHaMLYp1p9qg96jeGBgdc_zNuWhs'

# Predefined Tickers (74 Tickers)
TICKERS = [
    "SPY", "QQQ", "IWM", "GLD", "NVDA", "AAPL", "TSLA", "META", "PLTR", "GOOGL",
    "AMD", "MSFT", "AMZN", "NFLX", "AVGO", "CRWD", "SNOW", "ADBE", "CRM", "DDOG",
    "PANW", "MU", "QCOM", "NOW", "COIN", "MARA", "SQ", "PYPL", "SOFI", "HOOD",
    "AFRM", "CHWY", "COST", "PG", "CAVA", "CELH", "ABNB", "UBER", "LYFT", "DASH",
    "CVNA", "HD", "CAT", "DE", "CVX", "XOM", "WM", "RTX", "LLY", "JNJ", "UNH",
    "MRNA", "JPM", "SHOP", "BABA", "RBLX", "SNAP", "AI", "PATH", "UPST", "DKNG",
    "ZM", "RIVN", "DOCU", "GS", "MRNA", "REGN", "VRTX", "INCY", "BNTX", "SBUX",
    "BIDU", "DIS", "TGT", "ISRG"
]

signal_count = {t: {"CALL":0, "PUT":0} for t in TICKERS}

# ---------------------------------
# HELPER FUNCTIONS
# ---------------------------------

def fetch_alpha_vantage_equity_data(symbol, interval='daily', outputsize='full'):
    base_url = "https://www.alphavantage.co/query"
    if interval == 'daily':
        params = {
            "function": "TIME_SERIES_DAILY_ADJUSTED",
            "symbol": symbol,
            "apikey": ALPHA_VANTAGE_API_KEY,
            "outputsize": outputsize
        }
        r = requests.get(base_url, params=params).json()
        if "Time Series (Daily)" not in r:
            return pd.DataFrame()
        data = r["Time Series (Daily)"]
        df = pd.DataFrame(data).T
        df.columns = [c.split(". ")[1] for c in list(data[list(data.keys())[0]].keys())]
        df = df.rename(columns={
            'open': 'open',
            'high': 'high',
            'low': 'low',
            'close': 'close',
            'adjusted close': 'adjusted_close',
            'volume': 'volume',
            'dividend amount': 'dividend_amount',
            'split coefficient': 'split_coefficient'
        })
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        df[['open','high','low','close','volume']] = df[['open','high','low','close','volume']].astype(float)
        return df
    else:
        # For intraday intervals like '5min', '15min', '60min'
        params = {
            "function": "TIME_SERIES_INTRADAY",
            "symbol": symbol,
            "interval": interval,
            "apikey": ALPHA_VANTAGE_API_KEY,
            "outputsize": outputsize
        }
        r = requests.get(base_url, params=params).json()
        key = f"Time Series ({interval})"
        if key not in r:
            return pd.DataFrame()
        data = r[key]
        df = pd.DataFrame(data).T
        df.columns = [c.split(". ")[1] for c in list(data[list(data.keys())[0]].keys())]
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        df[['open','high','low','close','volume']] = df[['open','high','low','close','volume']].astype(float)
        return df

def fetch_alpha_vantage_options_data(symbol):
    # Alpha Vantage OPTION_CHAIN endpoint
    # https://www.alphavantage.co/documentation/#option-chain-data
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "OPTION_CHAIN",
        "symbol": symbol,
        "apikey": ALPHA_VANTAGE_API_KEY
    }
    r = requests.get(base_url, params=params).json()
    # Structure example:
    # {
    #   "symbol": "IBM",
    #   "data": [
    #     {
    #       "expirationDate": "2024-12-20",
    #       "calls": [ {...}, ...],
    #       "puts": [ {...}, ...]
    #     },
    #     ...
    #   ]
    # }
    if "data" in r:
        return r["data"]
    else:
        return []

def get_current_price(symbol):
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": "GLOBAL_QUOTE",
        "symbol": symbol,
        "apikey": ALPHA_VANTAGE_API_KEY
    }
    r = requests.get(base_url, params=params).json()
    if "Global Quote" not in r:
        return None
    price = float(r["Global Quote"]["05. price"])
    return price

def send_telegram_message(message):
    telegram_bot.send_message(chat_id=TELEGRAM_CHAT_ID, text=message, parse_mode='HTML')

def send_discord_message(message):
    webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=message)
    webhook.execute()

def log_to_google_sheets(data_row):
    sheet.append_row(data_row)

# ---------------------------------
# INDICATORS
# ---------------------------------

def EMA(series, period):
    return series.ewm(span=period, adjust=False).mean()

def RSI(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ema_up = up.ewm(com=period-1, adjust=False).mean()
    ema_down = down.ewm(com=period-1, adjust=False).mean()
    rs = ema_up / ema_down
    return 100 - (100/(1+rs))

def MACD(series, fast=12, slow=26, signal=9):
    fast_ema = EMA(series, fast)
    slow_ema = EMA(series, slow)
    macd_line = fast_ema - slow_ema
    signal_line = EMA(macd_line, signal)
    hist = macd_line - signal_line
    return macd_line, signal_line, hist

def ATR(df, period=14):
    high = df['high']
    low = df['low']
    close = df['close']
    hl = high - low
    hc = (high - close.shift()).abs()
    lc = (low - close.shift()).abs()
    tr = pd.DataFrame({'hl':hl, 'hc':hc, 'lc':lc}).max(axis=1)
    atr = tr.rolling(period).mean()
    return atr

def VWAP(df):
    tp = (df['high'] + df['low'] + df['close'])/3
    vwap = (tp * df['volume']).cumsum() / df['volume'].cumsum()
    return vwap

def BollingerBands(series, period=20, std_factor=2):
    sma = series.rolling(period).mean()
    std = series.rolling(period).std()
    upper = sma + std_factor*std
    lower = sma - std_factor*std
    return sma, upper, lower

def StochasticRSI(series, period=14, smoothK=3, smoothD=3):
    rsi_ = RSI(series, period)
    min_rsi = rsi_.rolling(period).min()
    max_rsi = rsi_.rolling(period).max()
    stoch_rsi = (rsi_ - min_rsi)/(max_rsi - min_rsi)
    K = stoch_rsi.rolling(smoothK).mean()
    D = K.rolling(smoothD).mean()
    return K, D

def CMF(df, period=20):
    mf_multiplier = ((df['close'] - df['low']) - (df['high'] - df['close']))/(df['high']-df['low'])
    mf_volume = mf_multiplier * df['volume']
    cmf = mf_volume.rolling(period).sum() / df['volume'].rolling(period).sum()
    return cmf

def add_indicators(df):
    df['RSI'] = RSI(df['close'],14)
    df['EMA_9'] = EMA(df['close'],9)
    df['EMA_21'] = EMA(df['close'],21)
    df['EMA_50'] = EMA(df['close'],50)
    macd_line, macd_signal, macd_hist = MACD(df['close'],12,26,9)
    df['MACD'] = macd_line
    df['MACD_Signal'] = macd_signal
    df['MACD_Hist'] = macd_hist
    df['ATR_14'] = ATR(df,14)
    df['VWAP'] = VWAP(df)
    df['BB_Mid'], df['BB_Upper'], df['BB_Lower'] = BollingerBands(df['close'],20,2)
    df['StochK'], df['StochD'] = StochasticRSI(df['close'],14,3,3)
    df['CMF'] = CMF(df,20)
    return df

# ---------------------------------
# OPTION CRITERIA
# ---------------------------------

def classify_option_type(current_price, strike_price):
    diff = abs(strike_price - current_price)
    if diff <= 0.05*current_price:
        return "ATM"
    elif 0.05*current_price < diff <= 0.15*current_price:
        return "OTM"
    elif 0.15*current_price < diff <= 0.30*current_price:
        return "Deep OTM"
    else:
        return "NONE"

def check_option_criteria(option_type, volume, delta, iv):
    if option_type == "ATM":
        return volume >= 500 and 0.30 <= delta <= 0.60 and iv <= 45
    elif option_type == "OTM":
        return volume >= 1000 and 0.10 <= delta <= 0.30 and iv <= 55
    elif option_type == "Deep OTM":
        return volume >= 2000 and 0.05 <= delta <= 0.15 and iv <= 65
    return False

def detect_gamma_squeeze(volume, avg_volume, open_interest, iv_change):
    if volume > 3*avg_volume and open_interest > 20000 and iv_change > 20:
        return True
    return False

# ---------------------------------
# SENTIMENT (Placeholder)
# Since Alpha Vantage does not provide sentiment, we simulate:
# For example, if last daily close > previous daily close, "Bullish",
# if lower, "Bearish", else "Neutral".
def approximate_sentiment(df):
    if df['close'].iloc[-1] > df['close'].iloc[-2]:
        return "Bullish"
    elif df['close'].iloc[-1] < df['close'].iloc[-2]:
        return "Bearish"
    else:
        return "Neutral"

# ---------------------------------
# SIGNAL FORMATTING
# ---------------------------------

def generate_signal_message(ticker, current_price, trade_type, strike_price, expiration_date,
                            premium_low, premium_high, volume, open_interest,
                            entry_low, entry_high, break_even, stop_loss_price, stop_loss_percentage,
                            tp_levels, explanation, timestamp):
    msg = f"☕ Coffee Talk Trade Signals ☕\n"
    msg += f"🎯 {ticker} Alert\n"
    msg += f" 💰 Current Price: ${current_price:.2f}\n"
    msg += f" 📈 Trade Type: {trade_type.upper()}\n"
    msg += f" 🎯 Strike Price: ${strike_price:.2f}\n"
    msg += f" 📅 Expiration Date: {expiration_date}\n"
    msg += f" 💵 Premium Range: ${premium_low:.2f} - ${premium_high:.2f}\n"
    msg += f" 📊 Volume: {volume}\n"
    msg += f" 🔄 Open Interest: {open_interest}\n"
    msg += f"⚠️ Risk Management:\n"
    msg += f" • Entry Zone: ${entry_low:.2f} - ${entry_high:.2f}\n"
    msg += f" • Break Even: ${break_even:.2f} (25% from Entry)\n"
    msg += f" • Stop Loss: ${stop_loss_price:.2f} (-{stop_loss_percentage}%)\n"
    msg += f" • Take Profit Levels:\n"
    for i, tp in enumerate(tp_levels, start=1):
        msg += f" {i}️⃣ ${tp:.2f}\n"
    msg += f"📌 Need to Know:\n"
    msg += f" • {explanation}\n"
    msg += f" • Highlights: Gamma squeeze, unusual volume, or key trend alignment.\n"
    msg += f"🔍 Verify all data points before entry.\n"
    msg += f" ⏰ {timestamp}\n"
    return msg

def backtest_strategy():
    # Placeholder
    pass

def daily_report():
    # Placeholder
    pass

def pre_market_analysis():
    # Placeholder pre-market analysis using Alpha Vantage broad indicators or just neutral
    return "Neutral"

# ---------------------------------
# MAIN LOGIC
# ---------------------------------

def main():
    pre_market_sentiment = pre_market_analysis()

    now = datetime.datetime.now()
    end_date = now.strftime("%Y-%m-%d")
    # For daily data, just get last 60 days:
    start_date = (now - datetime.timedelta(days=60)).strftime("%Y-%m-%d")

    for ticker in TICKERS:
        try:
            current_price = get_current_price(ticker)
            if current_price is None:
                continue

            # Fetch daily data for indicators
            daily_df = fetch_alpha_vantage_equity_data(ticker, 'daily', 'full')
            if daily_df.empty or len(daily_df)<30:
                continue
            daily_df = add_indicators(daily_df)

            # Fetch intraday data for multi-timeframe analysis:
            # 4H approximation: get 1H data and resample
            oneh_df = fetch_alpha_vantage_equity_data(ticker, '60min', 'full')
            if oneh_df.empty:
                continue
            fourh_df = oneh_df.resample('4H').agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'}).dropna()
            # Add indicators if needed
            if len(fourh_df)>30:
                fourh_df = add_indicators(fourh_df)

            fifteen_df = fetch_alpha_vantage_equity_data(ticker, '15min', 'full')
            if len(fifteen_df)>30:
                fifteen_df = add_indicators(fifteen_df)

            five_df = fetch_alpha_vantage_equity_data(ticker, '5min', 'full')
            if len(five_df)>30:
                five_df = add_indicators(five_df)

            # Approximate sentiment from price action
            sentiment = approximate_sentiment(daily_df)

            # Determine direction from sentiment
            direction = "CALL" if sentiment == "Bullish" else ("PUT" if sentiment == "Bearish" else None)
            if direction is None:
                continue

            # Fetch options data
            options_chains = fetch_alpha_vantage_options_data(ticker)
            if not options_chains:
                continue

            selected_option = None
            # Attempt to find suitable option based on criteria
            for chain in options_chains:
                exp = chain.get("expirationDate")
                side_data = chain["calls"] if direction=="CALL" else chain["puts"]
                for opt in side_data:
                    strike = float(opt['strike'])
                    vol = float(opt['volume'])
                    delta = float(opt['delta'])
                    iv = float(opt['impliedVolatility'])*100
                    opt_type = classify_option_type(current_price, strike)
                    if check_option_criteria(opt_type, vol, delta, iv):
                        selected_option = {
                            "strike": strike,
                            "volume": vol,
                            "delta": delta,
                            "iv": iv,
                            "expiration": exp,
                            "bid": float(opt['bid']),
                            "ask": float(opt['ask']),
                            "openInterest": float(opt['openInterest'])
                        }
                        break
                if selected_option:
                    break

            if not selected_option:
                continue

            # Frequency checks
            total_signals = signal_count[ticker]["CALL"] + signal_count[ticker]["PUT"]
            if total_signals >= 2:
                continue
            if total_signals == 1:
                prev_dir = "CALL" if signal_count[ticker]["CALL"]>0 else "PUT"
                if prev_dir == direction:
                    continue

            # Risk management
            entry_low = current_price*0.995
            entry_high = current_price*1.005
            break_even = ((entry_low+entry_high)/2)*1.25
            stop_loss_percentage = 10
            stop_loss_price = ((entry_low+entry_high)/2)*(1 - stop_loss_percentage/100)
            if direction == "CALL":
                tp_levels = [current_price*1.10, current_price*1.20, current_price*1.30]
            else:
                tp_levels = [current_price*0.90, current_price*0.80, current_price*0.70]

            # Gamma squeeze approximation
            avg_volume_20d = daily_df['volume'].tail(20).mean()
            # IV change placeholder: assume current iv vs. a baseline of 30
            iv_change = selected_option["iv"] - 30
            gamma_squeeze = detect_gamma_squeeze(selected_option["volume"], avg_volume_20d, selected_option["openInterest"], iv_change)

            explanation = "This contract was selected due to the prevailing trend and recent price action confirming sentiment. Technical indicators and volatility conditions aligned to create a favorable setup. The option contract meets criteria for volume, delta, and implied volatility."

            timestamp = datetime.datetime.now().strftime("%d-%m-%Y %H:%M")

            signal_msg = generate_signal_message(
                ticker, current_price, direction, selected_option["strike"],
                selected_option["expiration"], selected_option["bid"], selected_option["ask"],
                selected_option["volume"], selected_option["openInterest"],
                entry_low, entry_high, break_even, stop_loss_price, stop_loss_percentage,
                tp_levels, explanation, timestamp
            )

            # Send signals
            send_telegram_message(signal_msg)
            send_discord_message(signal_msg)

            # Log to Google Sheets
            data_row = [
                str(datetime.datetime.now()),
                ticker,
                direction,
                selected_option["strike"],
                selected_option["expiration"],
                f"{selected_option['bid']}-{selected_option['ask']}",
                sentiment,
                daily_df['RSI'].iloc[-1],
                daily_df['MACD'].iloc[-1],
                daily_df['VWAP'].iloc[-1],
                gamma_squeeze,
                selected_option["volume"],
                selected_option["openInterest"],
                explanation
            ]
            log_to_google_sheets(data_row)

            # Update signal count
            signal_count[ticker][direction] += 1

        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    daily_report()
    backtest_strategy()

if __name__ == "__main__":
    main()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.7/662.7 kB 10.3 MB/s eta 0:00:00


In [10]:
from discord_webhook import DiscordWebhook
test_message = "Testing Discord webhook"
webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=test_message)
response = webhook.execute()
print(response)


<Response [200]>


In [16]:
from telegram import Bot
test_bot = Bot(token="7454430837:AAHFtVI3WJ-eG7UBUxAxGx6jkC4IfcghjQg")
resp = test_bot.send_message(chat_id="1001989070467", text="Test message")
print(resp)


<coroutine object Bot.send_message at 0x786c90270580>
